# MOSFET

## Metal-Oxide-Semiconductor Field-Effect Transistor

The MOSFET is the fundamental building block of modern digital electronics. Understanding its operation is essential for circuit design and device engineering.

**Learning Objectives:**
- Understand MOSFET structure and operation
- Simulate transfer characteristics (Id-Vg) with PADRE
- Simulate output characteristics (Id-Vd) with PADRE
- Extract threshold voltage and transconductance from simulation data

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## Device Parameters Reference

The `describe()` function shows all available parameters for the MOSFET factory, including geometry, doping, physical models, and sweep options.

In [ ]:
from nanohubpadre import Simulation

# Show all available parameters for the MOSFET
Simulation.describe('mosfet')

---

## 1. MOSFET Physics

### 1.1 Structure

```
     Gate (Metal/Poly)
     ================
     |   Gate Oxide  |
     ================
  Source    Channel    Drain
  (N+)    (Inversion)   (N+)
  ====     -------     ====
    |                    |
    |    P-substrate     |
    +--------------------+
           Body
```

### 1.2 Key Parameters

- **Threshold voltage** ($V_{th}$): Gate voltage to create inversion
  $$V_{th} = V_{FB} + 2\phi_F + \frac{\sqrt{2\epsilon_s q N_a (2\phi_F)}}{C_{ox}}$$

- **Drain current** (linear region, $V_{ds} < V_{gs} - V_{th}$):
  $$I_d = \mu_n C_{ox} \frac{W}{L} \left[(V_{gs} - V_{th})V_{ds} - \frac{V_{ds}^2}{2}\right]$$

- **Drain current** (saturation, $V_{ds} > V_{gs} - V_{th}$):
  $$I_d = \frac{1}{2} \mu_n C_{ox} \frac{W}{L} (V_{gs} - V_{th})^2$$

- **Transconductance**:
  $$g_m = \frac{\partial I_d}{\partial V_{gs}} = \mu_n C_{ox} \frac{W}{L} (V_{gs} - V_{th})$$

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nanohubpadre import create_mosfet

# Physical constants
q = 1.6e-19
eps_0 = 8.85e-14  # F/cm
eps_si = 11.7 * eps_0
eps_ox = 3.9 * eps_0
kT = 0.0259  # eV at 300K
ni = 1.5e10  # intrinsic carrier concentration (cm^-3)

# MOSFET parameters
L = 0.1e-4    # Channel length (cm) = 100nm
W = 1e-4      # Channel width (cm) = 1um
tox = 5e-7    # Oxide thickness (cm) = 5nm
Na = 1e18     # Substrate doping (cm^-3)
mu_n = 400    # Electron mobility (cm^2/V-s)

# Calculated parameters
Cox = eps_ox / tox  # Oxide capacitance (F/cm^2)
phi_F = kT * np.log(Na / ni)  # Fermi potential

# Long-channel threshold voltage (n+ poly gate on p-Si)
Vfb  = -0.56 - phi_F                                   # flat-band: n+ poly gate
Qd   = np.sqrt(2 * eps_si * q * Na * 2 * phi_F)        # max depletion charge density
Vth  = Vfb + 2 * phi_F + Qd / Cox                      # threshold voltage

def mosfet_theory(Vgs, Vds, Vth_val, L_cm, Cox_val=Cox):
    """Long-channel Id: linear + saturation (pinch-off at Vds = Vgs - Vth)."""
    Vov, Vds_b = np.broadcast_arrays(np.maximum(Vgs - Vth_val, 0.0), Vds)
    Vds_eff = np.minimum(Vds_b, Vov)
    return mu_n * Cox_val * (W / L_cm) * (Vov * Vds_eff - 0.5 * Vds_eff**2)

print("MOSFET Parameters:")
print("="*40)
print(f"Channel length: {L*1e4*1000:.0f} nm")
print(f"Oxide thickness: {tox*1e7:.0f} nm")
print(f"Cox = {Cox*1e6:.2f} μF/cm²")
print(f"phi_F = {phi_F:.3f} V")
print(f"Vth   = {Vth:.3f} V")

---

## 2. Creating and Running a MOSFET Simulation

Let's create an NMOS transistor and run PADRE to examine its characteristics.

In [ ]:
# Create NMOS transistor
sim_nmos = create_mosfet(
    # Geometry
    channel_length=0.1,         # 100nm gate length
    gate_oxide_thickness=0.005, # 5nm oxide
    junction_depth=0.02,        # 20nm junction depth
    device_width=0.125,         # Device width
    device_depth=0.068,         # Substrate depth
    
    # Mesh
    nx=51,
    ny=51,
    
    # Doping
    channel_doping=1e18,        # Channel doping (p-type for NMOS)
    substrate_doping=5e16,      # Substrate doping
    source_drain_doping=1e20,   # N+ S/D doping
    device_type='nmos',
    
    # Models
    temperature=300,
    bgn=True,                   # Band-gap narrowing
    
    # Output - enable band diagrams
    log_bands_eq=True
)

# Visualize the device structure
sim_nmos.device_schematic()

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_nmos.generate_deck())

In [ ]:
# Run equilibrium simulation
print("Running MOSFET equilibrium simulation...")
result_eq = sim_nmos.run()

if result_eq.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_eq.stderr}")


---

## 3. Transfer Characteristics (Id-Vg)

The transfer characteristic shows drain current vs gate voltage at fixed drain voltage. Let's simulate this with PADRE.

In [ ]:
# Create MOSFET with transfer characteristic sweep
sim_idvg = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    junction_depth=0.02,
    device_width=0.125,
    device_depth=0.068,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='nmos',
    
    # Models
    temperature=300,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="idvg",
    
    # Transfer characteristic: sweep Vgs at fixed Vds
    vgs_sweep=(0.0, 1.5, 0.05),  # Vgs: 0 to 1.5V
    vds=0.1                       # Vds = 0.1V (linear region)
)

print("Transfer Characteristic Simulation Configuration:")
print("="*40)
print("Gate voltage sweep: 0V to 1.5V")
print("Drain voltage: 0.1V (linear region)")
print(f"Number of bias points: {int(1.5/0.05) + 1}")

In [ ]:
# Run the transfer characteristic simulation
print("Running transfer characteristic simulation...")
print("(This may take a few minutes)")
result_idvg = sim_idvg.run()

if result_idvg.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_idvg.stderr}")


In [ ]:
# Plot the simulated transfer characteristic
try:
    # Get I-V data from simulation
    iv_data = sim_idvg.get_iv_data()
    
    # Gate electrode is typically electrode 3, drain is electrode 2
    Vgs_sim, Id_sim = iv_data.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
    
    Vgs_sim = np.array(Vgs_sim)
    Id_sim = np.array(Id_sim)
    
    # Create figure with two subplots: linear (left) and log (right)
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Transfer Characteristic (Linear Scale)', 'Transfer Characteristic (Log Scale)'))
    
    # Linear scale plot (left panel)
    fig.add_trace(go.Scatter(
        x=Vgs_sim, y=Id_sim * 1e6,
        mode="lines+markers",
        line=dict(color="blue", width=2),
        marker=dict(size=4),
        name="PADRE Simulation"
    ), row=1, col=1)
    
    # Log scale plot (right panel) – showlegend=False (legend already in linear panel)
    fig.add_trace(go.Scatter(
        x=Vgs_sim, y=Id_sim,
        mode="lines+markers",
        line=dict(color="blue", width=2),
        marker=dict(size=4),
        name="PADRE Simulation",
        showlegend=False
    ), row=1, col=2)
    
    # Log y-axis on right panel
    fig.update_yaxes(type="log", row=1, col=2)
    
    # Axis labels
    fig.update_xaxes(title_text="Gate Voltage Vgs (V)", row=1, col=1)
    fig.update_yaxes(title_text="Drain Current Id (μA)", row=1, col=1)
    fig.update_xaxes(title_text="Gate Voltage Vgs (V)", row=1, col=2)
    fig.update_yaxes(title_text="Drain Current Id (A)", row=1, col=2)
    
    # Annotation for subthreshold region on the log panel
    fig.add_annotation(text="Subthreshold<br>region", x=0.2, y=1e-10, showarrow=False, font=dict(size=11), row=1, col=2)
    

    # Long-channel theory overlay (linear panel only)
    Vgs_th = np.linspace(0, 1.5, 200)
    Id_th  = mosfet_theory(Vgs_th, 0.1, Vth, L)
    fig.add_trace(go.Scatter(
        x=Vgs_th, y=Id_th * 1e6, mode="lines",
        name="Long-channel model",
        line=dict(color="gray", width=2, dash="dash"), opacity=0.6
    ), row=1, col=1)

    fig.update_layout(template="plotly_white", width=1000, height=450)
    fig.show()
    
    print("\nSimulated Transfer Characteristic:")
    print("="*40)
    print(f"Vgs range: {Vgs_sim[0]:.2f}V to {Vgs_sim[-1]:.2f}V")
    print(f"Id range: {Id_sim.min():.3e} A to {Id_sim.max():.3e} A")
    
except Exception as e:
    print(f"Could not plot transfer characteristic: {e}")
    print("Trying built-in plot method...")
    sim_idvg.plot_transfer(gate_electrode=3, drain_electrode=2)

In [ ]:
# Extract threshold voltage from simulation
try:
    # Method 1: Linear extrapolation
    # Find max gm point (maximum slope)
    gm = np.gradient(Id_sim, Vgs_sim)
    max_gm_idx = np.argmax(gm)
    
    if max_gm_idx > 0 and max_gm_idx < len(Id_sim) - 1:
        slope = gm[max_gm_idx]
        intercept = Id_sim[max_gm_idx] - slope * Vgs_sim[max_gm_idx]
        Vth_linear = -intercept / slope if slope > 0 else np.nan
    else:
        Vth_linear = np.nan
    
    # Method 2: Constant current method (Ith = 100nA)
    I_th = 1e-7  # 100 nA
    idx_above = np.where(Id_sim > I_th)[0]
    if len(idx_above) > 0:
        Vth_const = Vgs_sim[idx_above[0]]
    else:
        Vth_const = np.nan
    
    # Calculate transconductance
    gm_max = np.max(gm) * 1e6  # μS
    
    # Calculate subthreshold swing
    # SS = dVgs / d(log10(Id))
    mask = (Id_sim > 1e-12) & (Id_sim < 1e-7)
    if np.sum(mask) > 2:
        Vgs_sub = Vgs_sim[mask]
        Id_sub = Id_sim[mask]
        slope_sub = np.polyfit(np.log10(Id_sub), Vgs_sub, 1)[0]
        SS = slope_sub * 1000  # mV/decade
    else:
        SS = np.nan
    
    print("Extracted MOSFET Parameters:")
    print("="*40)
    print(f"Threshold voltage (linear extrap): Vth = {Vth_linear:.3f} V")
    print(f"Threshold voltage (constant I): Vth = {Vth_const:.3f} V")
    print(f"Peak transconductance: gm_max = {gm_max:.2f} μS")
    print(f"Subthreshold swing: SS = {SS:.1f} mV/decade")
    print(f"\nNote: Ideal SS at 300K = 60 mV/decade")
    
except Exception as e:
    print(f"Could not extract parameters: {e}")

---

## 4. Output Characteristics (Id-Vd)

The output characteristic shows drain current vs drain voltage at different gate voltages.

In [ ]:
# Create MOSFET with output characteristic sweep
sim_idvd = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    junction_depth=0.02,
    device_width=0.125,
    device_depth=0.068,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='nmos',
    
    temperature=300,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="idvd",
    
    # Output characteristic: sweep Vds at fixed Vgs
    vds_sweep=(0.0, 1.5, 0.05),  # Vds: 0 to 1.5V
    vgs=1.0                       # Vgs = 1.0V
)

print("Output Characteristic Simulation Configuration:")
print("="*40)
print("Drain voltage sweep: 0V to 1.5V")
print("Gate voltage: 1.0V (fixed)")

In [ ]:
# Run the output characteristic simulation
print("Running output characteristic simulation...")
result_idvd = sim_idvd.run()

if result_idvd.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_idvd.stderr}")


In [ ]:
# Plot the simulated output characteristic
try:
    iv_data = sim_idvd.get_iv_data()
    Vds_sim, Id_out = iv_data.get_output_characteristic(drain_electrode=2)
    
    Vds_sim = np.array(Vds_sim)
    Id_out = np.array(Id_out)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=Vds_sim, y=Id_out * 1e6,
        mode="lines+markers",
        line=dict(color="blue", width=2),
        marker=dict(size=4),
        name="Vgs = 1.0V"
    ))
    
    # Long-channel theory overlay
    Vds_th = np.linspace(0, 1.5, 200)
    Id_th  = mosfet_theory(1.0, Vds_th, Vth, L)
    fig.add_trace(go.Scatter(
        x=Vds_th, y=Id_th * 1e6, mode="lines",
        name="Long-channel model",
        line=dict(color="gray", width=2, dash="dash"), opacity=0.6
    ))

    # Mark linear and saturation regions
    Vdsat = 1.0 - Vth
    
    # Vertical dashed line at Vdsat
    fig.add_vline(x=Vdsat, line_dash="dash", line_color="red", opacity=0.5)
    
    # Region annotations
    fig.add_annotation(text="Linear", x=0.15, y=Id_out.max() * 0.5e6, showarrow=False, font=dict(size=11))
    fig.add_annotation(text="Saturation", x=1.0, y=Id_out.max() * 0.5e6, showarrow=False, font=dict(size=11))
    
    fig.update_xaxes(title_text="Drain Voltage Vds (V)")
    fig.update_yaxes(title_text="Drain Current Id (μA)")
    fig.update_layout(
        title_text="NMOS Output Characteristic (PADRE Simulation)",
        template="plotly_white",
        width=1000,
        height=450
    )
    fig.show()
    
except Exception as e:
    print(f"Could not plot output characteristic: {e}")

In [ ]:
# Simulate output characteristics at multiple gate voltages
Vgs_values = [0.6, 0.8, 1.0, 1.2]
output_results = {}

print("Simulating output characteristics at multiple Vgs:")
print("="*50)

for Vgs in Vgs_values:
    print(f"\nVgs = {Vgs}V...")
    
    sim = create_mosfet(
        channel_length=0.1,
        gate_oxide_thickness=0.005,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"idvd_vgs{int(Vgs*10)}",
        vds_sweep=(0.0, 1.5, 0.05),
        vgs=Vgs
    )
    
    result = sim.run()
    
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed:\n{result.stderr}")
    try:
        iv = sim.get_iv_data()
        Vds, Id = iv.get_output_characteristic(drain_electrode=2)
        output_results[Vgs] = (np.array(Vds), np.array(Id))
        print(f"  Success")
    except:
        print(f"  Could not parse data")


In [ ]:
# Plot family of output characteristics
if output_results:
    fig = go.Figure()
    
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    
    for i, (Vgs, (Vds, Id)) in enumerate(output_results.items()):
        color = colors[i % len(colors)]
        fig.add_trace(go.Scatter(
            x=Vds, y=Id * 1e6,
            mode="lines",
            line=dict(color=color, width=2),
            name=f'Vgs = {Vgs}V'
        ))

        # Long-channel theory overlay (same color, dashed)
        Vds_th = np.linspace(0, 1.5, 200)
        Id_th  = mosfet_theory(Vgs, Vds_th, Vth, L)
        fig.add_trace(go.Scatter(
            x=Vds_th, y=Id_th * 1e6, mode="lines",
            name=f'Theory Vgs={Vgs}V',
            line=dict(color=color, width=2, dash="dash"),
            opacity=0.5, showlegend=False
        ))
    
    fig.update_xaxes(title_text="Drain Voltage Vds (V)")
    fig.update_yaxes(title_text="Drain Current Id (μA)")
    fig.update_layout(
        title_text="NMOS Output Characteristics (PADRE Simulation)",
        template="plotly_white",
        width=1000,
        height=450
    )
    fig.show()
    
    print("\nOutput characteristic analysis:")
    print("- Linear region: Id increases with Vds")
    print("- Saturation region: Id relatively constant (channel pinch-off)")
    print("- Higher Vgs → Higher saturation current")

---

## 5. Channel Length Effects

Let's explore how channel length affects transistor behavior using PADRE simulations.

In [ ]:
# Compare different channel lengths
channel_lengths = [0.2, 0.1, 0.05]  # microns
length_results = {}

print("Channel Length Comparison:")
print("="*50)

for L_um in channel_lengths:
    print(f"\nL = {L_um*1000:.0f} nm...")
    
    sim = create_mosfet(
        channel_length=L_um,
        gate_oxide_thickness=0.005,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"idvg_L{int(L_um*1000)}nm",
        vgs_sweep=(0.0, 1.5, 0.05),
        vds=0.1
    )
    
    result = sim.run()
    
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed:\n{result.stderr}")
    try:
        iv = sim.get_iv_data()
        Vgs, Id = iv.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
        length_results[L_um] = (np.array(Vgs), np.array(Id))
        print(f"  Success")
    except:
        print(f"  Could not parse data")


In [ ]:
# Plot channel length comparison
if length_results:
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Transfer Characteristic (Linear)', 'Transfer Characteristic (Log)'))
    
    colors = ['blue', 'green', 'red']
    
    for i, (L_um, (Vgs, Id)) in enumerate(length_results.items()):
        color = colors[i % len(colors)]
        label = f'L = {L_um*1000:.0f} nm'
        
        # Linear panel (left)
        fig.add_trace(go.Scatter(
            x=Vgs, y=Id * 1e6,
            mode="lines",
            line=dict(color=color, width=2),
            name=label
        ), row=1, col=1)
        
        # Log panel (right) – showlegend=False
        fig.add_trace(go.Scatter(
            x=Vgs, y=Id,
            mode="lines",
            line=dict(color=color, width=2),
            name=label,
            showlegend=False
        ), row=1, col=2)

        # Long-channel theory overlay (linear panel, same color, dashed)
        Vgs_th = np.linspace(0, 1.5, 200)
        Id_th  = mosfet_theory(Vgs_th, 0.1, Vth, L_um * 1e-4)
        fig.add_trace(go.Scatter(
            x=Vgs_th, y=Id_th * 1e6, mode="lines",
            name=f'Theory L={L_um*1000:.0f}nm',
            line=dict(color=color, width=2, dash="dash"),
            opacity=0.5, showlegend=False
        ), row=1, col=1)
    
    # Log y-axis on right panel
    fig.update_yaxes(type="log", row=1, col=2)
    
    # Axis labels
    fig.update_xaxes(title_text="Vgs (V)", row=1, col=1)
    fig.update_yaxes(title_text="Id (μA)", row=1, col=1)
    fig.update_xaxes(title_text="Vgs (V)", row=1, col=2)
    fig.update_yaxes(title_text="Id (A)", row=1, col=2)
    
    fig.update_layout(template="plotly_white", width=1000, height=450)
    fig.show()
    
    print("\nKey observations:")
    print("- Shorter channel → Higher current (Id ~ 1/L)")
    print("- Short-channel effects may reduce Vth (DIBL)")
    print("- Subthreshold slope may degrade at very short L")

---

## 6. DIBL Analysis (Drain-Induced Barrier Lowering)

DIBL is a short-channel effect where the drain voltage affects the threshold voltage.

In [ ]:
# Simulate at low and high Vds to measure DIBL
Vds_low = 0.05
Vds_high = 1.0
dibl_results = {}

print("DIBL Analysis:")
print("="*50)

for Vds in [Vds_low, Vds_high]:
    print(f"\nVds = {Vds}V...")
    
    sim = create_mosfet(
        channel_length=0.1,
        gate_oxide_thickness=0.005,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"dibl_vds{int(Vds*100)}",
        vgs_sweep=(0.0, 1.5, 0.02),
        vds=Vds
    )
    
    result = sim.run()
    
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed:\n{result.stderr}")
    try:
        iv = sim.get_iv_data()
        Vgs, Id = iv.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
        dibl_results[Vds] = (np.array(Vgs), np.array(Id))
        print(f"  Success")
    except:
        print(f"  Could not parse data")

In [ ]:
# Plot DIBL comparison
if len(dibl_results) == 2:
    Vgs_low, Id_low = dibl_results[Vds_low]
    Vgs_high, Id_high = dibl_results[Vds_high]
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=Vgs_low, y=Id_low,
        mode="lines",
        line=dict(color="blue", width=2),
        name=f'Vds = {Vds_low}V'
    ))
    
    fig.add_trace(go.Scatter(
        x=Vgs_high, y=Id_high,
        mode="lines",
        line=dict(color="red", width=2),
        name=f'Vds = {Vds_high}V'
    ))
    
    # Log y-axis
    fig.update_yaxes(type="log")
    
    fig.update_xaxes(title_text="Gate Voltage Vgs (V)")
    fig.update_yaxes(title_text="Drain Current Id (A)")
    fig.update_layout(
        title_text="DIBL Analysis (PADRE Simulation)",
        template="plotly_white",
        width=1000,
        height=450
    )
    fig.show()
    
    # Extract Vth at constant current and calculate DIBL
    I_th = 1e-7  # 100 nA
    
    idx_low = np.where(Id_low > I_th)[0]
    idx_high = np.where(Id_high > I_th)[0]
    
    if len(idx_low) > 0 and len(idx_high) > 0:
        Vth_low = Vgs_low[idx_low[0]]
        Vth_high = Vgs_high[idx_high[0]]
        
        delta_Vth = Vth_low - Vth_high
        delta_Vds = Vds_high - Vds_low
        DIBL = delta_Vth / delta_Vds * 1000  # mV/V
        
        print(f"\nDIBL Analysis Results:")
        print("="*40)
        print(f"Vth at Vds={Vds_low}V: {Vth_low:.3f} V")
        print(f"Vth at Vds={Vds_high}V: {Vth_high:.3f} V")
        print(f"ΔVth = {delta_Vth*1000:.1f} mV")
        print(f"DIBL = {DIBL:.1f} mV/V")
        print(f"\nNote: DIBL < 100 mV/V is typically acceptable")

---

## 7. Oxide Thickness Effect

Gate oxide thickness affects Cox, Vth, and overall device performance.

In [ ]:
# Compare different oxide thicknesses
tox_values = [0.002, 0.005, 0.010]  # 2nm, 5nm, 10nm
tox_results = {}

print("Oxide Thickness Comparison:")
print("="*50)

for tox in tox_values:
    print(f"\ntox = {tox*1000:.0f} nm...")
    Cox_calc = eps_ox / (tox * 1e-4)  # F/cm^2
    print(f"  Cox = {Cox_calc*1e6:.2f} μF/cm²")
    
    sim = create_mosfet(
        channel_length=0.1,
        gate_oxide_thickness=tox,
        channel_doping=1e18,
        source_drain_doping=1e20,
        device_type='nmos',
        temperature=300,
        log_iv=True,
        iv_file=f"tox_{int(tox*1000)}nm",
        vgs_sweep=(0.0, 1.5, 0.05),
        vds=0.1
    )
    
    result = sim.run()
    
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed:\n{result.stderr}")
    try:
        iv = sim.get_iv_data()
        Vgs, Id = iv.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
        tox_results[tox] = (np.array(Vgs), np.array(Id))
        print(f"  Simulation success")
    except:
        print(f"  Could not parse data")

In [ ]:
# Plot oxide thickness comparison
if tox_results:
    fig = go.Figure()
    
    colors = ['blue', 'green', 'red']
    
    for i, (tox, (Vgs, Id)) in enumerate(tox_results.items()):
        color = colors[i % len(colors)]
        label = f'tox = {tox*1000:.0f} nm'
        fig.add_trace(go.Scatter(
            x=Vgs, y=Id * 1e6,
            mode="lines",
            line=dict(color=color, width=2),
            name=label
        ))

        # Long-channel theory (Cox and Vth both depend on tox)
        Vgs_th  = np.linspace(0, 1.5, 200)
        Cox_t   = eps_ox / (tox * 1e-4)
        Vth_t   = Vfb + 2 * phi_F + Qd / Cox_t
        Id_th   = mosfet_theory(Vgs_th, 0.1, Vth_t, L, Cox_val=Cox_t)
        fig.add_trace(go.Scatter(
            x=Vgs_th, y=Id_th * 1e6, mode="lines",
            name=f'Theory tox={tox*1000:.0f}nm',
            line=dict(color=color, width=2, dash="dash"),
            opacity=0.5, showlegend=False
        ))
    
    fig.update_xaxes(title_text="Gate Voltage Vgs (V)")
    fig.update_yaxes(title_text="Drain Current Id (μA)")
    fig.update_layout(
        title_text="Effect of Oxide Thickness (PADRE Simulation)",
        template="plotly_white",
        width=1000,
        height=450
    )
    fig.show()
    
    print("\nKey observations:")
    print("- Thinner oxide → Higher Cox → Higher current")
    print("- Thinner oxide → Better gate control")
    print("- Trade-off: Thinner oxide increases gate leakage")

---

## 8. 2D Contour Maps

Heat maps of the electrostatic potential, carrier concentrations, and electric field across the MOSFET cross-section reveal the spatial distribution of the channel, depletion regions, and how these change under gate and drain bias.

PADRE's `PLOT.3D` command dumps mesh-node data (x, y, z, value) as a scatter file. We interpolate that scattered data onto a regular grid using `scipy.interpolate.griddata`, then overlay Plotly heat maps with contour lines.

We compare two conditions:
- **Equilibrium** ($V_{gs} = V_{ds} = 0$): potential set by doping profiles and gate workfunction.
- **On-state bias** ($V_{gs} = 1.0\,\text{V}$, $V_{ds} = 1.0\,\text{V}$): inversion layer formed, current flowing.

In [ ]:
# Run a simulation that saves Plot3D scatter files at equilibrium and on-state bias
# The contour_maps=True parameter automatically adds PLOT.3D commands to the deck

sim_2d = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    junction_depth=0.02,
    device_width=0.125,
    device_depth=0.068,
    channel_doping=1e18,
    substrate_doping=5e16,
    source_drain_doping=1e20,
    device_type='nmos',
    temperature=300,
    bgn=True,
    # Enable 2D contour maps: dumps Plot3D scatter files at equilibrium and on-state
    contour_maps=True,
    contour_vgs=1.0,    # Gate voltage for on-state
    contour_vds=1.0,    # Drain voltage for on-state
)

print("Running simulation for 2D contour maps...")
print("  Equilibrium: pot, dop, el, hh, ef, qfn, qfp")
print("  Final bias (Vgs=1V, Vds=1V): pot, el, hh, ef, qfn, qfp")
result_2d = sim_2d.run()
if result_2d.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_2d.stderr}")


In [ ]:
# ── 1. Doping Profile ──
sim_2d.plot_contour("dop_eq",
                    title="Doping Profile (log10 scale)",
                    colorscale="Jet", log_scale=True,
                    cbar_title="Doping (cm^-3)");

In [ ]:
# ── 2. Potential: Equilibrium vs On-state ──
sim_2d.plot_contour(["pot_eq", "pot_bias"],
                    title=["Potential — Equilibrium",
                           "Potential — Vgs=1V, Vds=1V"],
                    cbar_title="V");

In [ ]:
# ── 3. Electron Concentration: Equilibrium vs On-state ──
sim_2d.plot_contour(["el_eq", "el_bias"],
                    title=["Electron Conc. — Equilibrium (log10)",
                           "Electron Conc. — Vgs=1V, Vds=1V (log10)"],
                    colorscale="Blues", log_scale=True,
                    cbar_title="n (cm^-3)");

print("Note: Under bias, the inversion layer (high electron concentration)")
print("appears under the gate oxide — this is the conducting channel.")

In [ ]:
# ── 4. Hole Concentration: Equilibrium vs On-state ──
sim_2d.plot_contour(["hh_eq", "hh_bias"],
                    title=["Hole Conc. — Equilibrium (log10)",
                           "Hole Conc. — Vgs=1V, Vds=1V (log10)"],
                    colorscale="Reds", log_scale=True,
                    cbar_title="p (cm^-3)");

print("Note: Holes are depleted under the gate in both conditions.")
print("In the substrate, hole concentration matches the doping level.")

In [ ]:
# ── 5. Quasi-Fermi Levels at Final Bias ──
sim_2d.plot_contour(["qfn_bias", "qfp_bias"],
                    title=["Electron Quasi-Fermi Level (Vgs=1V, Vds=1V)",
                           "Hole Quasi-Fermi Level (Vgs=1V, Vds=1V)"],
                    colorscale="Viridis",
                    cbar_title="QF (V)");

print("Note: The electron quasi-Fermi level gradient along the channel")
print("drives the drain current. The hole quasi-Fermi level is approximately")
print("flat in the substrate (no hole current flowing).")

In [ ]:
# ── 6. Electric Field: Equilibrium vs On-state ──
sim_2d.plot_contour(["ef_eq", "ef_bias"],
                    title=["Electric Field — Equilibrium",
                           "Electric Field — Vgs=1V, Vds=1V"],
                    colorscale="Hot", log_scale=True,
                    cbar_title="E (V/cm)");

print("Note: The electric field is highest at the gate oxide interface")
print("and at the drain-side junction under bias (hot carrier region).")

---

## 9. Exercises

### Exercise 1: PMOS Transistor
Create a PMOS transistor and compare its characteristics to the NMOS.

In [ ]:
# Exercise 1: PMOS simulation
sim_pmos = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='pmos',  # PMOS
    temperature=300,
    log_iv=True,
    iv_file="pmos_idvg",
    vgs_sweep=(0.0, -1.5, -0.05),  # Negative voltages for PMOS
    vds=-0.1                        # Negative Vds for PMOS
)

print("Running PMOS simulation...")
result_pmos = sim_pmos.run()

if result_pmos.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_pmos.stderr}")
print("PMOS simulation completed!")

try:
    iv_pmos = sim_pmos.get_iv_data()
    Vgs_p, Id_p = iv_pmos.get_transfer_characteristic(gate_electrode=3, drain_electrode=2)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=np.abs(Vgs_p), y=np.abs(Id_p) * 1e6,
        mode="lines+markers",
        line=dict(color="red", width=2),
        marker=dict(size=4),
        name="PMOS"
    ))
    
    fig.update_xaxes(title_text="|Vgs| (V)")
    fig.update_yaxes(title_text="|Id| (μA)")
    fig.update_layout(
        title_text="PMOS Transfer Characteristic",
        template="plotly_white",
        width=1000,
        height=450
    )
    fig.show()
    
    print("\nPMOS vs NMOS:")
    print("- PMOS uses holes as carriers (lower mobility)")
    print("- PMOS current ~2-3x lower than NMOS at same |Vgs|")
    print("- PMOS turns on with negative Vgs")
    
except Exception as e:
    print(f"Could not plot: {e}")


---

## Summary

In this notebook, you learned:

1. **MOSFET Structure**: Source, drain, gate, and channel regions
2. **Running PADRE Simulations**: Using `create_mosfet()` and `sim.run()`
3. **Transfer Characteristics**: Id vs Vgs from PADRE simulation
4. **Output Characteristics**: Linear and saturation regions
5. **Parameter Extraction**: Vth, gm, SS from simulated data
6. **Short-Channel Effects**: DIBL analysis from simulation
7. **Design Parameters**: Effect of channel length and oxide thickness
8. **2D Contour Maps**: Doping, potential, carrier concentrations, quasi-Fermi levels, and electric field

**Key Equations:**
- Linear region: $I_d = \mu C_{ox}\frac{W}{L}[(V_{gs}-V_{th})V_{ds} - \frac{V_{ds}^2}{2}]$
- Saturation: $I_d = \frac{1}{2}\mu C_{ox}\frac{W}{L}(V_{gs}-V_{th})^2$

**Next**: [05 - BJT](05_BJT.ipynb) - Bipolar Junction Transistors